In [1]:
import os
import time
import random
import time
import requests 
import re
import logging
import subprocess
from subprocess import Popen
from sys import platform
import os, sys
import logging
import json
import threading
import numpy as np

import subprocess
import time
import signal
from typing import Dict

from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node
from netunicorn.base.task import Failure, Task, TaskDispatcher


from netunicorn.client.remote import RemoteClient, RemoteClientException
from netunicorn.base import Experiment, ExperimentStatus, Pipeline
from netunicorn.library.tasks.basic import SleepTask
from netunicorn.library.tasks.measurements.ookla_speedtest import SpeedTest
from netunicorn.library.tasks.measurements.ping import Ping
from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node
from netunicorn.base.task import Failure, Task, TaskDispatcher
from netunicorn.base import Result, Failure, Success, Task, TaskDispatcher
from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node

from typing import Dict
from typing import Optional
from enum import IntEnum
from datetime import datetime

from returns.pipeline import is_successful
from returns.result import Failure

In [3]:
import signal
import subprocess
import time
from typing import List, Optional

from netunicorn.base import (
    Architecture,
    Failure,
    Node,
    Result,
    Success,
    Task,
    TaskDispatcher,
)
from netunicorn.library.tasks.tasks_utils import subprocess_run


class StartCapture(Task):
    def __init__(self, arguments=None):
        super().__init__()
        self.arguments = arguments or ["-i", "eth0", "tcp"]

    def run(self) -> Result:
        try:
            # Capture packets using tcpdump and store output in memory
            proc = subprocess.Popen(
                ["tcpdump", *self.arguments, "-U", "-w", "-"],  # Write to stdout (-w -)
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE
            )
            stdout_data, stderr_data = proc.communicate(timeout=10)  # Capture data for 10 seconds

            if proc.returncode != 0:
                return Failure(f"tcpdump failed: {stderr_data.decode('utf-8')}")

            # Encode captured data in base64 to handle binary data as a string
            encoded_data = base64.b64encode(stdout_data).decode('utf-8')
            return Success(encoded_data)
        except Exception as e:
            return Failure(f"Error capturing packets: {str(e)}")


class StartCaptureLinuxImplementation(Task):
    requirements = ["sudo apt-get update", "sudo apt-get install -y tcpdump"]

    def __init__(
        self, filepath: str, arguments: Optional[List[str]] = None, *args, **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.arguments = arguments or []
        self.filepath = filepath

    def run(self) -> Result:
        signal.signal(signal.SIGCHLD, signal.SIG_IGN)

        proc = subprocess.Popen(
            ["tcpdump"] + self.arguments + ["-U", "-w", self.filepath],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
        )
        time.sleep(2)
        if (exit_code := proc.poll()) is None:  # not finished yet
            return Success(proc.pid)

        text = ""
        if proc.stdout:
            text += proc.stdout.read().decode("utf-8") + "\n"
        if proc.stderr:
            text += proc.stderr.read().decode("utf-8")
        return Failure(f"Tcpdump terminated with return code {exit_code}" + text)


class StopNamedCapture(TaskDispatcher):
    def __init__(self, start_capture_task_name: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.start_capture_task_name = start_capture_task_name
        self.linux_implementation = StopNamedCaptureLinuxImplementation(
            capture_task_name=self.start_capture_task_name,
            *args,
            **kwargs,
        )

    def dispatch(self, node: Node) -> Task:
        if node.architecture in {Architecture.LINUX_AMD64, Architecture.LINUX_ARM64}:
            return self.linux_implementation

        raise NotImplementedError(
            f"StopCapture is not implemented for {node.architecture}"
        )


class StopNamedCaptureLinuxImplementation(Task):
    requirements = [
        "sudo apt-get update",
        "sudo apt-get install -y tcpdump",
        "sudo apt-get install -y procps",
    ]

    def __init__(self, capture_task_name: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.capture_task_name = capture_task_name

    def run(self):
        signal.signal(signal.SIGCHLD, signal.SIG_IGN)
        pid = self.previous_steps.get(
            self.capture_task_name, [Failure("Named StartCapture not found")]
        )[-1]
        if isinstance(pid, Failure):
            return pid

        pid = pid.unwrap()
        return subprocess_run(["kill", str(pid)])


class StopAllTCPDumps(TaskDispatcher):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.linux_implementation = StopAllTCPDumpsLinuxImplementation(*args, **kwargs)

    def dispatch(self, node: Node) -> Task:
        if node.architecture in {Architecture.LINUX_AMD64, Architecture.LINUX_ARM64}:
            return self.linux_implementation

        raise NotImplementedError(
            f"StopAllTCPDumps is not implemented for {node.architecture}"
        )


class StopAllTCPDumpsLinuxImplementation(Task):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def run(self):
        signal.signal(signal.SIGCHLD, signal.SIG_IGN)
        return subprocess_run(["killall", "-w", "tcpdump"])

Code with tcp variant control

Edit the code using fileio for data.

In [ ]:
from netunicorn.base import Pipeline, Task
from netunicorn.client.remote import RemoteClient, RemoteClientException
from netunicorn.library.tasks.upload.fileio import UploadToFileIO
import time


class SetTCPVariant(Task):
    def __init__(self, variant: str):
        super().__init__()
        self.variant = variant

    def run(self):
        import subprocess
        command = f"sudo sysctl -w net.ipv4.tcp_congestion_control={self.variant}"
        result = subprocess.run(command, shell=True)
        if result.returncode == 0:
            return f"TCP congestion control set to {self.variant}"
        else:
            return f"Failed to set TCP congestion control to {self.variant}"


class StartCapture(Task):
    def __init__(self, filepath: str, arguments: list = None):
        super().__init__()
        self.filepath = filepath
        self.arguments = arguments or ["-i", "eth0", "tcp"]

    def run(self):
        import subprocess
        command = ["tcpdump", *self.arguments, "-c", "100", "-w", self.filepath]
        process = subprocess.Popen(command)
        return f"Started tcpdump with PID {process.pid}"


class StopCapture(Task):
    def run(self):
        import subprocess
        command = "sudo pkill tcpdump"
        result = subprocess.run(command, shell=True)
        if result.returncode == 0:
            return "tcpdump process terminated successfully"
        else:
            return "Failed to terminate tcpdump process"

netunicorn_login = "cs190n3"
netunicorn_password = "PXnLibpf"
NETUNICORN_ENDPOINT = os.environ.get('NETUNICORN_ENDPOINT', 'https://pinot.cs.ucsb.edu/netunicorn')
NETUNICORN_LOGIN = os.environ.get('NETUNICORN_LOGIN', netunicorn_login)
NETUNICORN_PASSWORD = os.environ.get('NETUNICORN_PASSWORD', netunicorn_password)

client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)

pipeline = (
    Pipeline()
    .then(SetTCPVariant(variant="cubic"))
    .then(StartCapture(filepath="/tmp/tcp_packets.pcap"))  
    .then(StopCapture()) 
    .then(UploadToFileIO(filepath="/tmp/tcp_packets.pcap")) 
)

experiment_label = "cubic_tcp_capture"
working_node = "snl-server-5"  
nodes = client.get_nodes()
working_nodes = nodes.filter(lambda node: node.name.startswith(working_node)).take(1)

experiment = Pipeline().map(pipeline, working_nodes)

try:
    client.delete_experiment(experiment_label)
except RemoteClientException:
    pass

client.prepare_experiment(experiment, experiment_label)
while True:
    status = client.get_experiment_status(experiment_label)
    print(f"Experiment Status: {status.status}")
    if status.status == "READY":
        break
    time.sleep(5)

client.start_execution(experiment_label)
while True:
    status = client.get_experiment_status(experiment_label)
    print(f"Experiment Status: {status.status}")
    if status.status != "RUNNING":
        break
    time.sleep(10)

print(f"Experiment {experiment_label} completed!")

In [ ]:
from netunicorn.base import Pipeline, Task
from netunicorn.client.remote import RemoteClient, RemoteClientException
from netunicorn.library.tasks.upload.fileio import UploadToFileIO
import time


class SetTCPVariant(Task):
    def __init__(self, variant: str):
        super().__init__()
        self.variant = variant

    def run(self):
        import subprocess
        command = f"sudo sysctl -w net.ipv4.tcp_congestion_control={self.variant}"
        result = subprocess.run(command, shell=True)
        if result.returncode == 0:
            return f"TCP congestion control set to {self.variant}"
        else:
            return f"Failed to set TCP congestion control to {self.variant}"


class StartCapture(Task):
    def __init__(self, filepath: str, arguments: list = None):
        super().__init__()
        self.filepath = filepath
        self.arguments = arguments or ["-i", "eth0", "tcp"]

    def run(self):
        import subprocess
        command = ["tcpdump", *self.arguments, "-c", "100", "-w", self.filepath]
        process = subprocess.Popen(command)
        return f"Started tcpdump with PID {process.pid}"


class StopCapture(Task):
    def run(self):
        import subprocess
        command = "sudo pkill tcpdump"
        result = subprocess.run(command, shell=True)
        if result.returncode == 0:
            return "tcpdump process terminated successfully"
        else:
            return "Failed to terminate tcpdump process"

netunicorn_login = "cs190n3"
netunicorn_password = "PXnLibpf"
NETUNICORN_ENDPOINT = os.environ.get('NETUNICORN_ENDPOINT', 'https://pinot.cs.ucsb.edu/netunicorn')
NETUNICORN_LOGIN = os.environ.get('NETUNICORN_LOGIN', netunicorn_login)
NETUNICORN_PASSWORD = os.environ.get('NETUNICORN_PASSWORD', netunicorn_password)

client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)

for i in range(1, 101):
    pipeline = (
        Pipeline()
        .then(SetTCPVariant(variant="cubic"))
        .then(StartCapture(filepath="/tmp/tcp_packets.pcap"))  
        .then(StopCapture()) 
        .then(UploadToFileIO(filepath="/tmp/tcp_packets.pcap")) 
    )

experiment_label = "cubic_tcp_capture"
working_node = "snl-server-5"  
nodes = client.get_nodes()
working_nodes = nodes.filter(lambda node: node.name.startswith(working_node)).take(1)

experiment = Pipeline().map(pipeline, working_nodes)

try:
    client.delete_experiment(experiment_label)
except RemoteClientException:
    pass

client.prepare_experiment(experiment, experiment_label)
while True:
    status = client.get_experiment_status(experiment_label)
    print(f"Experiment Status: {status.status}")
    if status.status == "READY":
        break
    time.sleep(5)

client.start_execution(experiment_label)
while True:
    status = client.get_experiment_status(experiment_label)
    print(f"Experiment Status: {status.status}")
    if status.status != "RUNNING":
        break
    time.sleep(10)

print(f"Experiment {experiment_label} completed!")